In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch,torchvision
print(f"torch version：{torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")
print(f"CUDA Version: {torch.version.cuda}")
print(f"cuDNN version is: {torch.backends.cudnn.version()}")

torch version：1.11.0+cu113
torchvision version: 0.12.0+cu113
CUDA Version: 11.3
cuDNN version is: 8200


In [ ]:
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision.transforms as T

In [ ]:
!pip install pytorchvideo av
!pip install av
import pytorchvideo
import av
print(av.__version__)

9.2.0


In [ ]:
import torch

In [ ]:
FOLDERNAME = 'CS_231n_Project/ConvLSTM'
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My\ Drive/$FOLDERNAME

/content/drive/My Drive/CS_231n_Project/ConvLSTM


In [ ]:
from torchvision.datasets import UCF101

# These are some minimal variables used to configure and load the dataset:
ucf_data_dir = "../UCF101/UCF-101"
ucf_label_dir = "../UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist"
frames_per_clip = 5
step_between_clips = 1
batch_size = 32

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader

import io
import imageio
from ipywidgets import widgets, HBox

# Use GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
tfs = T.Compose([
    # TODO: this should be done by a video-level transfrom when PyTorch provides transforms.ToTensor() for video
    # scale in [0, 1] of type float
    T.Lambda(lambda x: x / 255.),
    # reshape into (T, C, H, W) for easier convolutions
    T.Lambda(lambda x: x.permute(3, 0, 1, 2)),
    # rescale to the most common size
    T.Lambda(lambda x: nn.functional.interpolate(x, (240, 320))),
])

In [ ]:
# def custom_collate(batch):
#     filtered_batch = []
#     for video, _, label in batch:
#         filtered_batch.append((video, label))
#     return torch.utils.data.dataloader.default_collate(filtered_batch)

In [ ]:
def custom_collate(batch):

    # Add channel dim, scale pixels between 0 and 1, send to GPU
    filtered_batch = []
    for video, _, _ in batch:
      filtered_batch.append(video)

    batch = torch.tensor(filtered_batch).unsqueeze(1)     
    batch = batch / 255.0                        
    batch = batch.to(device)                     

    # Randomly pick 4 frames as input, 5th frame is target
    rand = np.random.randint(5,10)                     
    return batch[:,:,rand-5:rand], batch[:,:,rand]     


In [ ]:
# create train loader (allowing batches and other extras)
train_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                       step_between_clips=step_between_clips, train=True, transform=tfs)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                                           collate_fn=custom_collate)
# create test loader (allowing batches and other extras)
test_dataset = UCF101(ucf_data_dir, ucf_label_dir, frames_per_clip=frames_per_clip,
                      step_between_clips=step_between_clips, train=False, transform=tfs)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True,
                                          collate_fn=custom_collate)

  0%|          | 0/833 [00:00<?, ?it/s]

FileNotFoundError: ignored

In [ ]:
# Get a batch
input, _ = next(iter(test_loader))

# Reverse process before displaying
input = input.cpu().numpy() * 255.0     

for video in input.squeeze(1)[:3]:          # Loop over videos
    with io.BytesIO() as gif:
        imageio.mimsave(gif,video.astype(np.uint8),"GIF",fps=5)
        display(HBox([widgets.Image(value=gif.getvalue())]))

In [ ]:

model = Seq2Seq(num_channels=3, num_kernels=64, 
kernel_size=(3, 3), padding=(1, 1), activation="relu", 
frame_size=(64, 64), num_layers=3).to(device)

optim = Adam(model.parameters(), lr=1e-4)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.BCELoss(reduction='sum')

In [ ]:
num_epochs = 20

for epoch in range(1, num_epochs+1):
    
    train_loss = 0                                                 
    model.train()                                                  
    for batch_num, (input, target) in enumerate(train_loader, 1):  
        output = model(input)                                     
        loss = criterion(output.flatten(), target.flatten())       
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()                                           
        train_loss += loss.item()                                 
    train_loss /= len(train_loader.dataset)                       

    val_loss = 0                                                 
    model.eval()                                                   
    with torch.no_grad():                                          
        for input, target in test_loader:                          
            output = model(input)                                   
            loss = criterion(output.flatten(), target.flatten())   
            val_loss += loss.item()                                
    val_loss /= len(test_loader.dataset)                            

    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n".format(
        epoch, train_loss, val_loss))
    
    num_epochs = 20

for epoch in range(1, num_epochs+1):
    
    train_loss = 0                                                 
    model.train()                                                  
    for batch_num, (input, target) in enumerate(train_loader, 1):  
        output = model(input)                                     
        loss = criterion(output.flatten(), target.flatten())       
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()                                           
        train_loss += loss.item()                                 
    train_loss /= len(train_loader.dataset)                       

    val_loss = 0                                                 
    model.eval()                                                   
    with torch.no_grad():                                          
        for input, target in test_loader:                          
            output = model(input)                                   
            loss = criterion(output.flatten(), target.flatten())   
            val_loss += loss.item()                                
    val_loss /= len(test_loader.dataset)                            

    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n".format(
        epoch, train_loss, val_loss))

In [ ]:
def collate_test(batch):

    # Last 5 frames are target
    target = np.array(batch)[:,5:]                     
    
    # Add channel dim, scale pixels between 0 and 1, send to GPU
    batch = torch.tensor(batch).unsqueeze(1)          
    batch = batch / 255.0                             
    batch = batch.to(device)                          
    return batch, target

# Test Data Loader
# test_loader = DataLoader(test_data,shuffle=True, 
#                          batch_size=3, collate_fn=collate_test)

# Get a batch
batch, target = next(iter(test_loader))

# Initialize output sequence
output = np.zeros(target.shape, dtype=np.uint8)

# Loop over timesteps
for timestep in range(target.shape[1]):
  input = batch[:,:,timestep:timestep+10]   
  output[:,timestep]=(model(input).squeeze(1).cpu()>0.5)*255.0

In [ ]:
for tgt, out in zip(target, output):       # Loop over samples
    
    # Write target video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, tgt, "GIF", fps = 5)    
        target_gif = gif.getvalue()

    # Write output video as gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, out, "GIF", fps = 5)    
        output_gif = gif.getvalue()

    display(HBox([widgets.Image(value=target_gif), 
                  widgets.Image(value=output_gif)]))